In [1]:
import json
import pandas as pd
from unidecode import unidecode
import re

with open('data/subset_epita.json') as f:
    data = json.load(f)
    
def get_docs_from_id(id):
    docs = []
    for o in data:
        if o['document_type_id'] == id:
            docs.append(o)
    return docs

datas = get_docs_from_id(8)

def preprocess(word):
    lower = str.lower(word)
    normalized = unidecode(lower)
    punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ '''
    for ele in normalized:
        if ele in punc:
            normalized = normalized.replace(ele, "")
    return normalized

# Récupérer les mots par paragraphe

In [2]:
def abs(a, b):
    r = a - b
    if r < 0:
        return -r
    return r

def regrouper_mots_par_hauteur(doc, tolerance):
    mots_regroupes = []
    
    keys = list(doc.keys())
    keys.pop()
    keys.pop()
    for key in keys:
        page = {}
        lines = doc[key]
        for line in lines:
            for w in line['line']['words']:
                hauteur = w['top']
                found = False
                for h in page:
                    if abs(h, hauteur) <= tolerance:
                        found = True
                        page[h].append(w['text'])
                        break
                if found == True:
                    continue
                if hauteur not in page:
                    page[hauteur] = []
                page[hauteur].append(w['text'])
        mots_regroupes.append(page)
    return mots_regroupes

tolerance = 0.02
docs = []

for doc in datas:
    resultat = regrouper_mots_par_hauteur(doc, tolerance)
    docs.append(resultat)

# Afficher les groupes de mots
doc = docs[4]
for page in doc:
    print("@@@@@@@@@@@@@@@@")
    for hauteur, mots in page.items():
        print(mots)

@@@@@@@@@@@@@@@@
['Mme', 'Schneider', 'Stéphanie']
['ML', 'ab', 'Dr', 'Ph.', 'VINCENT', 'BIOPLUS', 'Pharmacien', 'biologiste', 'Enregistré', 'n°:', '773706437']
['9', 'rue', 'de', 'la', 'Faïencerie', '77130', 'MONTEREAU', 'Tél.', '01.64.32.18.61', 'Fax', '01.64.32.60.38']
['AU', 'LABORATOIRE']
['Mme', 'Schneider', 'Stéphanie', '56,', 'chemin', 'Virginie', 'Pasquier']
['77130', 'MAROLLES', 'SUR', 'SEINE']
['Prescripteur', ':', 'Dr', 'BOURIOT', 'MARINE', '73', 'RUE', 'DE', 'MONTREUIL', '75011', 'PARIS']
['Patient', ':', 'Mme', 'Schneider', 'Stéphanie', 'Né(e)', 'le', '21.07.1989', 'Née', 'Schneider']
['Dossier', 'N°', '9AFA051022', '-', '02', 'reçu', 'le', '20.02.2019', 'à', '08:43', 'Prélevé', 'le', '20.02.2019', 'à', '08:41', 'Edité', 'le', '06.03.2019', 'à', '17:47']
['Identifiant', "d'accès", 'à', 'vos', 'résultats', ':', 'P-001243621']
['MICROBIOLOGIE']
['Examens', 'ci-dessous', 'réalisés', 'le', '22.02.2019', ',', 'validés', 'le', '22.02.2019']
['PRELEVEMENT', 'URO-GENITAL']
['Orig

# Récupérer le top des unités trouvées dans les documents

In [3]:
with open("utils/results/bioloinc_units.json", 'r') as fichier_json:
    units = json.load(fichier_json)

In [4]:
units

{'%  réactivité (avec DTT)': ['%'],
 '%  réactivité (sans DTT)': ['%'],
 '% CD10': ['%'],
 '% CD16+CD56+': ['%'],
 '% CD19+': ['%'],
 '% CD3': ['%'],
 '% CD3+CD4+CD8': ['%'],
 '% CD3-CD16+CD56+': ['%'],
 '% CD3-CD19+': ['%'],
 '% CD4': ['%'],
 '% CD45': ['%'],
 '% CD5': ['%'],
 '% CD8': ['%'],
 "% d'hématies foetales": ['%', '%{RBCs}'],
 "% d'hémoglobine foetale": ["% d'Hb", '%{Hb}'],
 '% de réactivité IgG anti-HLA classe 1': ['%'],
 '% de réactivité IgG anti-HLA classe 2': ['%'],
 '1,25 dihydroxy vitamines D': ['pmol/L', 'pg/mL'],
 '1-hydroxymidazolam': ['ng/mL'],
 '1-méthylhistidine': ['µmol/L', 'umol/L'],
 '1-méthylhistidine/créatinine': ['µmol/mmol créat', 'umol/mmol{creat}'],
 '11-déoxycortisol': ['nmol/L', 'ng/mL'],
 '11-déoxycortisol J01 après métopirone': ['nmol/L'],
 '11-déoxycortisol avant métopirone': ['nmol/L'],
 '17-hydroxyprogestérone': ['nmol/L', 'ng/mL'],
 '17-hydroxyprogestérone salivaire': ['nmol/L'],
 '17-hydroxyprégnénolone': ['nmol/L', 'ng/mL'],
 '2-aminoadipate': 

In [11]:
# Récupérer la liste des marqueurs avec le nombre d'occurences
top_120_df = pd.read_csv('utils/results/top_120.csv')

# Garder que les marqueurs qui sont apparus au moins une fois
top_120_df = top_120_df.drop('Unnamed: 0', axis=1)[top_120_df['occurence'] >= 1]
top_120_list = list(top_120_df['grandeur'])

# Modifier le dictionnaire des unités pour garder que ceux qui apparaissent
all_units = set()
units_found = {}
for key in units :
    normalized_key = preprocess(key)
    for u in units[key]:
        all_units.add(u.lower())
    if key in top_120_list:
        units_found[normalized_key] = units[key]

all_units.add('T/L')
all_units.add('g/100ml')
all_units

{'%',
 "% [arb'u]/ml",
 "% d'hb",
 '%{hb}',
 '%{rbcs}',
 '/',
 '/10*3',
 '/10*3{rbcs}',
 '/10000 érythro',
 '/ml',
 '/ul',
 '/µl',
 '10*12/l',
 '10*3 cell/µl',
 '10*3/ul',
 '10*6/ml',
 '10*9/l',
 'T/L',
 "[apl'u]",
 "[arb'u]",
 "[arb'u]/ml",
 "[beth'u]",
 '[cfu]',
 "[gpl'u]",
 '[iu]/(24.h)',
 '[iu]/l',
 '[iu]/ml',
 'a',
 'ans',
 'cel',
 'cm',
 'copies/ml',
 'd',
 'f',
 'fl',
 'g',
 'g/(24.h)',
 'g/100ml',
 'g/24h',
 'g/dl',
 'g/g',
 'g/g{creat}',
 'g/l',
 'g/mmol',
 'h',
 'i',
 'j',
 'jours',
 'kg',
 'ku/l',
 'kui/l',
 'l',
 'l/l',
 'l/min',
 'log copies/ml',
 'log ui/ml',
 'm',
 'm2',
 'm[iu]/l',
 'm[iu]/ml',
 'mg/(24.h)',
 'mg/24h',
 'mg/dl',
 'mg/l',
 'mg/l{rbcs}',
 'mg/mg créat',
 'mg/mg{creat}',
 'mg/mmol créat',
 'mg/mmol{creat}',
 'min',
 'ml',
 'ml/min',
 'ml/min/1,73m²',
 'ml/min/{1.73_m2}',
 'mm',
 'mm hg',
 'mm[hg]',
 'mmol/(24.h)',
 'mmol/24h',
 'mmol/l',
 'mmol/mmol',
 'mmol/mmol{creat}',
 'mmol/mol',
 'mosm/l',
 'mosmol/l',
 'mu/l',
 'mu/ml',
 'mui/l',
 'mui/ml',
 'n',
 '

# Itérer sur les paragraphes pour trouver les marqueurs

In [17]:
def search_marqueur(words, start):
    for i in range(start, len(words)):
        normalized_word = preprocess(words[i])
        if normalized_word in units_found.keys():
            return i, normalized_word
    return -1, None

def search_unit(words, start, marqueur):
    for i in range(start + 1, len(words)):
            if words[i].lower() in units_found[marqueur]:
                return i, words[i]
    return -1, None    

def search_other_unit(words, start, marqueur):
    for i in range(start + 1, len(words)):
        if words[i].lower() in all_units :
            units_found[marqueur] += words[i]
            unit_in_doc = words[i]
            index_unit = i
            return words[i], i
    return None, -1

def search_value(words, start, end):
    for i in range(start + 1, end):
        if bool(re.match("^[0-9., <>]+$", words[i])) == True:
            return words[i]
    # Si la valeur n'est pas entre le marqueur et l'unité, on vérifie si la valeur est après l'unité
    if bool(re.match("^[0-9., <>]+$", words[end + 1])) == True:
        return words[end + 1]
    else :
        return None

def extract_values(doc):
    """
        doc : list of dict
    """
    values_found = {}
    for page in doc :
        for words in page.values():
            # Etape 1 : trouver le marqueur
            index_marqueur, marqueur = search_marqueur(words, 0)
            while index_marqueur != -1 :
            
                # Etape 2 : trouver l'unité avec bioloinc
                index_unit, unit_in_doc = search_unit(words, index_marqueur, marqueur)

                # Etape 3 : si l'unité ne correspond pas alors on va chercher dans la liste des unités
                if index_unit == -1 :
                    unit_in_doc, index_unit = search_other_unit(words, index_marqueur, marqueur)
                    if index_unit == -1 :
                        break
                        
                # Etape 4 : récupérer la valeur qui est entre le marqueur et l'unité
                value = search_value(words, index_marqueur, index_unit)
                if value == None:
                    break
                        
                # Etape 5 : si l'unité est % alors on vérifie s'il y a le mot 'soit' ou ':' juste après
                if unit_in_doc == '%' and (words[index_unit + 1] == 'soit' or words[index_unit + 1] == ':'):
                    index_unit, unit_in_doc = search_unit(words, index_unit+2, marqueur)
                    value = search_value(words, index_marqueur, index_unit)
                    if value == None or index_unit == -1 :
                        break

                # Etape 6 : stocker dans un dictionnaire
                values_found[marqueur] = value.replace(" ", "") + unit_in_doc
                
                # On relance pour vérifier s'il y a d'autres marqueurs sur la même ligne
                index_marqueur, marqueur = search_marqueur(words, index_unit + 1)

            
    return values_found

# Enlever les espaces dans les nombres
            
dictionnaire = extract_values(docs[5])
dictionnaire, len(dictionnaire.values())

({'leucocytes': '5,1G/L',
  'hemoglobine': '12,5%',
  'volume': '93fL',
  'tcmh': '31,3pg',
  'ccmh': '33,5g/dL',
  'idr': '13,0%',
  'lymphocytes': '30,6%',
  'monocytes': '9,8%',
  'plaquettes': '222G/L'},
 9)